# LASSO Regression (coordinate descent)

Optimize weights for Lasso regression (OLS + L1 penalty)

# Imports

In [1]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create is assigned to damiansp@gmail.com and will expire on March 07, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-27 19:28:50,206 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1464391728.log


# Copy useful functions from previous notebooks

In [3]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 
    features = ['constant'] + features 
    features_sframe = data_sframe[features]
    print features_sframe.head()

    feature_matrix = features_sframe.to_numpy()
    output_sarray = data_sframe[output]

    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [4]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    
    return(predictions)

# Normalize features
As with many ML algorithms, Lasso regression works best when features are normalized to all be on comparable scales.

A short function that normalizes columns of a given feature matrix. It also returns a pair `(normalized_features, norms)`, where the second item contains the norms of original features. This will be used to repeat identical transformation of test data.

In [5]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis = 0)
    normalized_features = feature_matrix / norms
    return (normalized_features, norms)    

Test:

In [6]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
print norms

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function (RSS + L1_penalty)
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, do not include `w[0]` in the L1 penalty term so as not to zero the intercept term.)

The absolute value sign makes the cost function non-differentiable, so gradient descent is not viable as it was for OLS or ridge regression.  Instead, this algorithm uses **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is:
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. 
Optimize one `w[i]` at a time, circling through the weights multiple times.  

Here **cyclical coordinate descent with normalized features** is used, where coordinates 0 to (d-1) are cycled through in order, and assumes the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (rho[i] + lambd a /2)     if rho[i] < -lambda / 2
w[i] = ├ 0                         if - lambda / 2 <= ro[i] <= lambda / 2
       └ (rho[i] - lambda / 2)     if rho[i] > lambda / 2
```
where
```
rho[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Do not regularize the intercept `w[0]`. It is updated as:
```
w[0] = rho[i]
```

## Effect of L1 penalty

Consider a simple model with 2 features:

In [7]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

+----------+-------------+----------+
| constant | sqft_living | bedrooms |
+----------+-------------+----------+
|    1     |    1180.0   |   3.0    |
|    1     |    2570.0   |   3.0    |
|    1     |    770.0    |   2.0    |
|    1     |    1960.0   |   4.0    |
|    1     |    1680.0   |   3.0    |
|    1     |    5420.0   |   4.0    |
|    1     |    1715.0   |   3.0    |
|    1     |    1060.0   |   3.0    |
|    1     |    1780.0   |   3.0    |
|    1     |    1890.0   |   3.0    |
+----------+-------------+----------+
[10 rows x 3 columns]



Normalize:

In [8]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
print simple_feature_matrix

[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]


Assign some random set of initial weights and inspect the values of `rho[i]`:

In [9]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [10]:
prediction = predict_output(simple_feature_matrix, weights)
prediction

array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

Compute the values of `rho[i]` for each feature in this simple model, using the formula given above, using the formula:
```
rho[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

In [11]:
rho = []

for i in xrange(3):
    feature_i = simple_feature_matrix[:, i] # 1 x n
    rho.append(sum(feature_i * (output - prediction + weights[i] * feature_i)))

In [12]:
rho1_zeros = (-2 * rho[1], 2 * rho[1])
rho2_zeros = (-2 * rho[2], 2 * rho[2])
print rho1_zeros
print rho2_zeros

(-175878941.54598159, 175878941.54598159)
(-161933397.3519305, 161933397.3519305)


So `rho[i]` quantifies the significance of the ith feature; the larger `rho[i]` is, the more likely it is for the ith feature to be retained.

## Single Coordinate Descent Step

Using the formula above, coordinate descent that minimizes the cost function over a single feature i is implemented. (Note that the intercept (weight 0) is not regularized.) Return updated weight for feature i.

In [13]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights) # (2,)
    
    # compute rho[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:, i] 
    rho_i = sum(feature_i * (output - prediction + weights[i] * feature_i))

    if i == 0: # intercept -- do not regularize
        new_weight_i = rho_i 
    elif rho_i < -l1_penalty / 2.:
        new_weight_i = rho_i + l1_penalty / 2
    elif rho_i > l1_penalty/2.:
        new_weight_i = rho_i - l1_penalty / 2
    else:
        new_weight_i = 0.
    
    return new_weight_i

Test

In [14]:
print lasso_coordinate_descent_step(
    i = 1, 
    feature_matrix = np.array([[3. / np.sqrt(13), 1./ np.sqrt(10)], [2. / np.sqrt(13), 3./ np.sqrt(10)]]),
    output = np.array([1., 1.]),
    weights = np.array([1., 4.]), 
    l1_penalty = 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, implement cyclical coordinate descent to optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When to stop? 

Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, stop.

In [15]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    max_step = 1e10
    weights = initial_weights
    
    while max_step > tolerance:
        max_step = 0
        
        for i in range(len(weights)):
            old_weight = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            
            coord_change = abs(weights[i] - old_weight)
            if coord_change > max_step:
                max_step = coord_change
            
    return weights

Set parameters to use:

In [16]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

Create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [17]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

+----------+-------------+----------+
| constant | sqft_living | bedrooms |
+----------+-------------+----------+
|    1     |    1180.0   |   3.0    |
|    1     |    2570.0   |   3.0    |
|    1     |    770.0    |   2.0    |
|    1     |    1960.0   |   4.0    |
|    1     |    1680.0   |   3.0    |
|    1     |    5420.0   |   4.0    |
|    1     |    1715.0   |   3.0    |
|    1     |    1060.0   |   3.0    |
|    1     |    1780.0   |   3.0    |
|    1     |    1890.0   |   3.0    |
+----------+-------------+----------+
[10 rows x 3 columns]



Run LASSO coordinate descent:

In [18]:
weights = lasso_cyclical_coordinate_descent(
    normalized_simple_feature_matrix, output, initial_weights, l1_penalty, tolerance)

In [19]:
print 'weights:', weights
preds = predict_output(normalized_simple_feature_matrix, weights)
error = preds - output
print 'rss:', sum(error ** 2)

weights: [ 21624998.36636353  63157246.78545319         0.        ]
rss: 1.63049248148e+15


# Evaluating LASSO fit with more features

Split the sales dataset into training and test sets.

In [20]:
train_data, test_data = sales.random_split(0.8, seed = 0)

Use the following set of features.

In [21]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

Create a normalized feature matrix from the TRAINING data with these features.  (Store the norms for the normalization, to be used again on the TEST data.)

In [22]:
(training_feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
train_normalized, train_norms = normalize_features(training_feature_matrix)

+----------+----------+-----------+-------------+----------+--------+------------+
| constant | bedrooms | bathrooms | sqft_living | sqft_lot | floors | waterfront |
+----------+----------+-----------+-------------+----------+--------+------------+
|    1     |   3.0    |    1.0    |    1180.0   |   5650   |   1    |     0      |
|    1     |   3.0    |    2.25   |    2570.0   |   7242   |   2    |     0      |
|    1     |   2.0    |    1.0    |    770.0    |  10000   |   1    |     0      |
|    1     |   4.0    |    3.0    |    1960.0   |   5000   |   1    |     0      |
|    1     |   3.0    |    2.0    |    1680.0   |   8080   |   1    |     0      |
|    1     |   4.0    |    4.5    |    5420.0   |  101930  |   1    |     0      |
|    1     |   3.0    |    2.25   |    1715.0   |   6819   |   2    |     0      |
|    1     |   3.0    |    1.5    |    1060.0   |   9711   |   1    |     0      |
|    1     |   3.0    |    1.0    |    1780.0   |   7470   |   1    |     0      |
|   

Learn the weights with `l1_penalty = 1e7`, on the training data.

In [29]:
init_weights = np.zeros(len(all_features) + 1)
tolerance = 1.0
#includes w[0], (intercept)
feat = ['intercept'] + all_features

In [30]:
weights1e7 = lasso_cyclical_coordinate_descent(
    train_normalized, output, init_weights, 1e7, tolerance)
print weights1e7

#for (f, w) in zip(feat, weights1e7):
#    print (f, w)

[ 24429600.60933346         0.                 0.          48389174.35227959
         0.                 0.           3317511.16271979
   7329961.98489638         0.                 0.                 0.
         0.                 0.                 0.        ]


## Rescaling learned weights

Because the weights were optimized for the *normalized* feature matrix, to use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:

Create a normalized version of each of the weights learned above.

In [31]:
weights1e7_normalized = weights1e7 / train_norms
print weights1e7
print weights1e7_normalized

[ 24429600.60933346         0.                 0.          48389174.35227959
         0.                 0.           3317511.16271979
   7329961.98489638         0.                 0.                 0.
         0.                 0.                 0.        ]
[  1.85285533e+05   0.00000000e+00   0.00000000e+00   1.61317456e+02
   0.00000000e+00   0.00000000e+00   2.87664700e+05   6.91937057e+04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]


Now, learn the weights with `l1_penalty = 1e8`, on the training data, then rescale.

In [33]:
weights1e8 = lasso_cyclical_coordinate_descent(
    train_normalized, output, init_weights, 1e8, tolerance)
print weights1e8
#for (f, w) in zip(feat, weights1e8):
#    print (f, w)

[ 71114625.75280942         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]


In [34]:
weights1e8_normalized = weights1e8 / train_norms
print weights1e8_normalized

[ 539366.62822135       0.               0.               0.               0.
       0.               0.               0.               0.               0.
       0.               0.               0.               0.        ]


And finally, learn the weights with `l1_penalty = 1e4`, on the training data, and rescale.

In [35]:
weights1e4 = lasso_cyclical_coordinate_descent(
    train_normalized, output, init_weights, 1e4, 5e5)
print weights1e4
#for (f, w) in zip(feat, weights1e4):
#    print (f, w)

[ 77779073.91265044 -22884012.25023264  15348487.08089934
  92166869.69883147  -2139328.08242773  -8818455.54409455
   6494209.73310656   7065162.05053201   4119079.21006769
  18436483.52618784 -14566678.54514407  -5528348.7517945  -83591746.20730424
   2784276.46012856]


In [36]:
weights1e4_normalized = weights1e4 / train_norms
print weights1e4
print weights1e4_normalized

[ 77779073.91265044 -22884012.25023264  15348487.08089934
  92166869.69883147  -2139328.08242773  -8818455.54409455
   6494209.73310656   7065162.05053201   4119079.21006769
  18436483.52618784 -14566678.54514407  -5528348.7517945  -83591746.20730424
   2784276.46012856]
[  5.89912924e+05  -4.97435039e+04   5.17044250e+04   3.07261390e+02
  -3.67765574e-01  -4.32048893e+04   5.63119400e+05   6.66940353e+04
   8.99767715e+03   1.80569342e+04  -5.60846894e+01  -7.88384489e+01
  -3.21603081e+02   5.18531810e+01]


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [39]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

+----------+----------+-----------+-------------+----------+--------+------------+
| constant | bedrooms | bathrooms | sqft_living | sqft_lot | floors | waterfront |
+----------+----------+-----------+-------------+----------+--------+------------+
|    1     |   3.0    |    1.0    |    1430.0   |  19901   |   1    |     0      |
|    1     |   4.0    |    3.0    |    2950.0   |   5000   |   2    |     0      |
|    1     |   3.0    |    2.0    |    1710.0   |   4697   |   1    |     0      |
|    1     |   3.0    |    2.5    |    2320.0   |   3980   |   2    |     0      |
|    1     |   3.0    |    1.0    |    1090.0   |   3000   |   1    |     0      |
|    1     |   4.0    |    2.5    |    2620.0   |   7553   |   2    |     0      |
|    1     |   4.0    |    2.25   |    4220.0   |  24186   |   1    |     0      |
|    1     |   4.0    |    2.5    |    2250.0   |   4495   |   2    |     0      |
|    1     |   3.0    |    1.75   |    1260.0   |   8400   |   1    |     0      |
|   

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [40]:
preds1e4 = predict_output(test_feature_matrix, weights1e4_normalized)
error = preds1e4 - test_output
rss1e4 = sum(error ** 2)
print 'rss:', rss1e4

rss: 2.2778100476e+14


In [41]:
preds1e7 = predict_output(test_feature_matrix, weights1e7_normalized)
error = preds1e7 - test_output
rss1e7 = sum(error ** 2)
print 'rss:', rss1e7

rss: 2.75962079909e+14


In [42]:
preds1e8 = predict_output(test_feature_matrix, weights1e8_normalized)
error = preds1e8 - test_output
rss1e8 = sum(error ** 2)
print 'rss:', rss1e8

rss: 5.37166150034e+14
